In [ ]:
import sys
import chipwhisperer as cw
from chipwhisperer.capture.targets.CW305 import CW305
import time
import numpy as np
from Crypto.Cipher import AES
import time
import matplotlib.pyplot as plt
import pickle



In [ ]:
proj = cw.open_project("projects/Default")


In [ ]:
proj = cw.open_project("projects/clx2_102708_118795_112045_51354_36250")


In [ ]:
#project_file = "projects/single_clock_no_random.cwp"
#proj = cw.create_project(project_file, overwrite=True)
proj = cw.import_project("projects/clx2_105529_fr_47717_114323_124716_137188.zip", overwrite=True)
proj.save()
x=[218,56,43, 161, 159, 221, 35, 30, 208, 37, 232, 130, 117, 38, 247, 91]
arr = bytearray(x)
key=arr

In [ ]:
traces = []
textin = []
textout = []
for (trace, pt, ct) in zip(proj.traces, proj.textins, proj.textouts):
    traces.append(trace.wave)
    textin.append(pt)
    textout.append(ct)
traces = np.array(traces)
textin = np.array(textin)
textout = np.array(textout)

In [ ]:
print(traces.shape, textin.shape, textout.shape)

In [ ]:
max_value = 0.35

mask = np.any(traces > max_value, axis=1)
traces = traces[~mask]
textin = textin[~mask]
textout = textout[~mask]

traces = np.array(traces)
textin = np.array(textin)
textout = np.array(textout)

#plt.plot(traces.T)
#plt.show()

In [ ]:
print(traces.shape, textin.shape, textout.shape)

In [ ]:
plt.plot(traces[:10000, :].T)
plt.show()

In [ ]:
import numpy as np
from tqdm import tnrange, trange

def synchronize_peaks(traces, threshold):
    
    num_traces = len(traces)
    num_samples = traces.shape[1]
    num_peaks = np.zeros(num_traces, dtype=int)
    peak_indices = np.zeros((num_traces, num_samples), dtype=bool)
    mask = np.full(num_traces, True)
    #mask[0] = False
    
    # Find peak indices for each trace
    for i in range(num_traces):
        temp = np.where((np.abs(traces[i]) > threshold))[0]

        prev = 0
        if len(temp) > 0:
            filtered_temp = [temp[0]]
            j = 0
            peak = []
            while j < len(temp) - 1:
                if temp[j] - prev >= 20:  # Check if the peak is at least 10 indices away from the previous peak
                    peak = [temp[j]]
                    prev = temp[j]
                    j += 1
                    while (temp[j] - prev < 10):
                        peak.append(temp[j])
                        if j < len(temp) - 1:
                            prev = temp[j]
                            j += 1
                        else:
                            break
                            
                    max_value = 100
                    max_index = None
                    for index in peak:
                        if traces[i][index] < max_value:
                            max_value = traces[i][index]
                            max_index = index
                    prev = max_index
                    filtered_temp.append(max_index)
                j += 1
                
            peak_indices[i, filtered_temp] = True
            num_peaks[i] = len(filtered_temp)

    synchronized_output = []
    # Find peak indices for the first trace
    first_trace_peaks = np.where(peak_indices[0])[0]

    #if len(first_trace_peaks) != 12:
        #return synchronized_output  # Return empty synchronized output if the first trace has fewer than 12 peaks

    # Synchronize peaks between traces
    for i in trange(0, num_traces):

        trace_peaks = []
        peak_index = np.where(peak_indices[i])[0]
        synchronized_peak = []
        #print(peak_index)
        if peak_index.shape[0] != 12:
                continue  # Break if the current trace has fewer than 12 peaks
        for j in range(12):

            temptrace = traces[i]
            
            #peak_diff = first_trace_peaks[j] - peak_index[j]
            #peak_diff = peak_diff%50
            #print(num_peaks[i])
            if peak_index[j] > traces.shape[1] - 40:
                synchronized_peak = []
                #mask[i] = False
                break
            if peak_index[j] < 10:
                rolled = np.roll(temptrace[peak_index[j] + 0:peak_index[j]+20], -10)
                synchronized_peak.append(rolled)

            else:
                rolled = np.roll(temptrace[peak_index[j] - 10:peak_index[j]+10], 0)
                synchronized_peak.append(rolled)
            
                #print(rolled, peak_index[j])
            #plt.plot(np.array(synchronized_peak).T)
            #plt.show()
        if len(synchronized_peak) > 1:
            synchronized_output.append(np.array(synchronized_peak).flatten())
            mask[i] = False
            #print(synchronized_peak)
            
    return (np.array(synchronized_output), mask)


In [ ]:
(peaks, mask) = synchronize_peaks(traces, 0.15)
#mask = np.insert(mask,0, True)
print(peaks.shape)
#print(traces.shape)

In [ ]:
peaktextin = textin[~mask]
peaktextout = textout[~mask]
print(peaktextin.shape, peaktextout.shape, peaks.shape)

In [ ]:
plt.plot(peaks[:1000].T)
plt.show()
#traces = peaks

In [ ]:
trimpeaks = peaks#[0:7000]
trimtextin = peaktextin#[0:7000]
trimtextout = peaktextout#[0:7000]

In [ ]:
peaks = peaks[:,:2200]

In [ ]:
proj = cw.create_project("projects/attack.cwp", overwrite=True)

for i in range(trimpeaks.shape[0]):
    trace = cw.Trace(trimpeaks[i], trimtextin[i], trimtextout[i], key)
    proj.traces.append(trace)
    #print(leak[i])
proj.save()

In [ ]:
import chipwhisperer as cw
import chipwhisperer.analyzer as cwa
project_file = "projects/attack"
project = cw.open_project(project_file)
attack = cwa.cpa(project, cwa.leakage_models.last_round_state_diff)
cb = cwa.get_jupyter_callback(attack)

In [ ]:
attack_results = attack.run(cb)

In [ ]:
from chipwhisperer.analyzer.attacks.models.aes.key_schedule import key_schedule_rounds
recv_lastroundkey = [kguess[0][0] for kguess in attack_results.find_maximums()]
recv_key = key_schedule_rounds(recv_lastroundkey, 10, 0)
for subkey in recv_key:
    print(hex(subkey))

In [ ]:
key = list(key)
assert (key == recv_key), "Failed to recover encryption key\nGot:      {}\nExpected: {}".format(recv_key, key)

In [ ]:
from chipwhisperer.analyzer.attacks.models.aes.key_schedule import key_schedule_rounds
import chipwhisperer as cw
import chipwhisperer.analyzer as cwa
import math

project_file = "projects/attack"
tracerange = 4500
step = 3000
modifier = 0
i = 0
modifiedRange = tracerange

while (i*step + modifiedRange) < peaks.shape[0]:
    
    proj = cw.create_project("projects/attack.cwp", overwrite=True)
    modifiedRange = tracerange - modifier
    
    currpeaks = peaks[i*step :i*step + modifiedRange]
    currtextin = peaktextin[i*step:i*step + modifiedRange]
    currtextout = peaktextout[i*step:i*step + modifiedRange]

    for j in range(currpeaks.shape[0]):
        trace = cw.Trace(currpeaks[j], currtextin[j], currtextout[j], key)
        proj.traces.append(trace)
    proj.save()

    #project = cw.open_project(project_file)
    attack = cwa.cpa(proj, cwa.leakage_models.last_round_state_diff)
    cb = cwa.get_jupyter_callback(attack)
    
    attack_results = attack.run()
    
    recv_lastroundkey = [kguess[0][0] for kguess in attack_results.find_maximums()]
    recv_key = key_schedule_rounds(recv_lastroundkey, 10, 0)

    key = list(key)
    
    if key == recv_key:
        print(recv_key, i, modifiedRange, len(currpeaks))
        modifier += 250
        i = 0
        continue
    print(i, modifiedRange, len(currpeaks))
    i += 1

In [ ]:
import math
out = [[]]
for j in range(traces.shape[0]):
    i = 0
    out.append([])
    while i < traces.shape[1]:
        val = traces[j][i]
        if val < -0.15:
            tempval = traces[j][i - 1]
            while(tempval >= val):
                i += 1
                if i >= traces.shape[1]:
                    break
                tempval = traces[j][i - 1]
                val = traces[j][i]
            #print(i)
            out[j].append(i)
            if len(out[j]) >= 12:
                break
            while val < 0:
                i += 1
                if i >= traces.shape[1]:
                    break
                val = traces[j][i]
        i += 1

d1 = [out[i][1:-1] for i in range(len(out))]

final = []
for i in range(len(d1)):
    for j in range(len(d1[i]) - 1):
        final.append(1/(d1[i][j+1] - d1[i][j]))
        
max_value = 0.05

filtered_list = [x*1000*(1/2*math.pi)*1.2 for x in final] # if x <= max_value]

plt.hist(filtered_list, bins=500)
plt.show()

In [ ]:
import math
out = [[]]
for i in range(traces.shape[0]):
    temp = np.where((np.abs(traces[i]) > 0.15))[0]

    prev = 0
    if len(temp) > 0:
        filtered_temp = [temp[0]]
        j = 0
        peak = []
        while j < len(temp) - 1:
            if temp[j] - prev >= 20:  # Check if the peak is at least 10 indices away from the previous peak
                peak = [temp[j]]
                prev = temp[j]
                j += 1
                while (temp[j] - prev < 10):
                    peak.append(temp[j])
                    if j < len(temp) - 1:
                        prev = temp[j]
                        j += 1
                    else:
                        break

                max_value = 100
                max_index = None
                for index in peak:
                    if traces[i][index] < max_value:
                        max_value = traces[i][index]
                        max_index = index
                prev = max_index
                filtered_temp.append(max_index)
            j += 1
        out.append(filtered_temp)
                

d1 = [out[i][1:-1] for i in range(len(out))]

final = []
for i in range(len(d1)):
    for j in range(len(d1[i]) - 1):
        final.append(1/(d1[i][j+1] - d1[i][j]))
        
max_value = 0.05

filtered_list = [x*1000*(1/2*math.pi) for x in final] # if x <= max_value]

plt.hist(filtered_list, bins=500)
plt.show()

In [ ]:
max_value = 0.015
filtered_list = [x*1000*(1/2*math.pi) for x in final ]#if x <= max_value]
plt.hist(filtered_list, bins=200, range=(0,70))
plt.show()